In [1]:
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [6]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    pairs = pairs[:100]
    print("Sampled %s sentence pairs (for faster training)" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    dataset = {}
    dataset["input_lang"] = input_lang
    dataset["output_lang"] = output_lang
    dataset["pairs"] = pairs
    return dataset

dataset = prepareData('eng', 'fra', True)
print(random.choice(dataset["pairs"]))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Sampled 100 sentence pairs (for faster training)
Counting words...
Counted words:
fra 92
eng 62
['je suis en forme .', 'i m fit .']


In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        # https://stackoverflow.com/a/48305882/8623609
        # encoder_output: give you the hidden layer outputs of the network for each time-step, but only for the final layer ("top")
        # encoder_hidden: give you the hidden layer outputs of the network for the last time-step only, but for all layers ("last right column")
        last_layer_encoder_hidden_states, last_time_step_encoder_hidden_states = self.gru(embedded, hidden)
        return last_layer_encoder_hidden_states, last_time_step_encoder_hidden_states

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderAttentionRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(DecoderAttentionRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        
        self.fc_hidden = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.fc_encoder = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.weight = nn.Parameter(torch.FloatTensor(1, hidden_size))
        self.attn_proj = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)        
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, decoder_input, decoder_hidden, last_layer_encoder_hidden_states_foreach_input):
        embedded = self.embedding(decoder_input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        x = torch.tanh(self.fc_hidden(decoder_hidden)+self.fc_encoder(last_layer_encoder_hidden_states_foreach_input))
        alignment_scores = x.bmm(self.weight.unsqueeze(2))
        attn_weights = F.softmax(alignment_scores.squeeze(2), dim=1)
        context_vector = torch.bmm(attn_weights.unsqueeze(0), last_layer_encoder_hidden_states_foreach_input.unsqueeze(0))

        output = torch.cat((embedded, context_vector), -1).squeeze(0)
        output = self.attn_proj(output).unsqueeze(0)
        output = F.relu(output)
        last_layer_decoder_hidden_states, last_time_step_decoder_hidden_states = self.gru(output, decoder_hidden)
        last_layer_decoder_hidden_states = F.log_softmax(self.out(last_layer_decoder_hidden_states[0]), dim=1)
        return last_layer_decoder_hidden_states, last_time_step_decoder_hidden_states, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    last_layer_encoder_hidden_states_foreach_input = torch.zeros(max_length, encoder.hidden_size, device=device)
    last_time_step_encoder_hidden_states = encoder.initHidden()
    loss = 0

    # https://stackoverflow.com/a/48305882/8623609
    # encoder_output: give you the hidden layer outputs of the network for each time-step, but only for the final layer ("top")
    # encoder_hidden: give you the hidden layer outputs of the network for the last time-step only, but for all layers ("last right column")
    for i in range(input_length):
        last_layer_encoder_hidden_states, last_time_step_encoder_hidden_states = encoder(input_tensor[i], last_time_step_encoder_hidden_states)
        last_layer_encoder_hidden_states_foreach_input[i] = last_layer_encoder_hidden_states.squeeze()

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = last_time_step_encoder_hidden_states

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for i in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, last_layer_encoder_hidden_states_foreach_input)
            loss += criterion(decoder_output, target_tensor[i])
            decoder_input = target_tensor[i]  # Teacher forcing
    else:
        # Without teacher forcing: use its own predictions as the next input
        for i in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, last_layer_encoder_hidden_states_foreach_input)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[i])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def trainIters(encoder, decoder, dataset, n_iters, print_every=100):
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
    training_pairs = [tensorsFromPair(dataset["input_lang"], dataset["output_lang"], random.choice(dataset["pairs"]))
                      for i in range(n_iters)]

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(f"Step: {iter} ({iter / n_iters * 100}%) Loss: {print_loss_avg}")

hidden_size = 256
encoder1 = EncoderRNN(dataset["input_lang"].n_words, hidden_size).to(device)
attn_decoder1 = DecoderAttentionRNN(hidden_size, dataset["output_lang"].n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, dataset, 1000)

Step: 100 (10.0%) Loss: 1.4960639825661983
Step: 200 (20.0%) Loss: 0.9305914251406988
Step: 300 (30.0%) Loss: 0.6985346862673761
Step: 400 (40.0%) Loss: 0.583538605988026
Step: 500 (50.0%) Loss: 0.30018163283665983
Step: 600 (60.0%) Loss: 0.2737010692954064
Step: 700 (70.0%) Loss: 0.2100257837921381
Step: 800 (80.0%) Loss: 0.13894890621801215
Step: 900 (90.0%) Loss: 0.1403068618948261
Step: 1000 (100.0%) Loss: 0.10856539232035482


In [11]:
def evaluate(encoder, decoder, dataset, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(dataset["input_lang"], sentence)
        input_length = input_tensor.size()[0]
        
        last_time_step_encoder_hidden_states = encoder.initHidden()
        last_layer_encoder_hidden_states_foreach_input = torch.zeros(max_length, encoder.hidden_size, device=device)

        for i in range(input_length):
            last_layer_encoder_hidden_states, last_time_step_encoder_hidden_states = encoder(input_tensor[i], last_time_step_encoder_hidden_states)
            last_layer_encoder_hidden_states_foreach_input[i] = last_layer_encoder_hidden_states.squeeze()

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = last_time_step_encoder_hidden_states

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for i in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, last_layer_encoder_hidden_states_foreach_input)
            print(decoder_output.shape)
            print(decoder_attention.shape)
            
            raise Exception
            decoder_attentions[i] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(dataset["output_lang"].index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:i + 1]

def evaluateRandomly(encoder, decoder, dataset, n=3):
    for i in range(n):
        pair = random.choice(dataset["pairs"])
        print("Input: ", pair[0])
        print("Expected:", pair[1])
        output_words, attentions = evaluate(encoder, decoder, dataset, pair[0])
        output_sentence = ' '.join(output_words)
        print("Pred: ", output_sentence)
        print('')

evaluateRandomly(encoder1, attn_decoder1, dataset)

Input:  je suis faineant .
Expected: i m lazy .
torch.Size([1, 62])
torch.Size([1, 10])


Exception: 